In [41]:
import os
import numpy as np
import pandas as pd

In [42]:
def read_excel(path):
    return pd.read_excel(path)

def delete_outlier(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]    
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_outlier = df[(df['SMP'] < lower_bound) | (df['SMP'] > upper_bound)]
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]
    
    # if(len(df_outlier)>0):
    #     print('Ada Outlier')

    # df_grouped = df_cleaned[['HARI', 'JAM', 'SMP']]
    # df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_outlier, df_cleaned


In [43]:
path_1 = '../Data/Preprocessing/Data Losarang/2024/2_input_losarang_2024.xlsx'

df = read_excel(path_1)
df.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-01 00:00:00,2024,1,1,Monday,00:00,78,30,12,120.0,15.6,30,30.0,76
1,2024-01-01 00:15:00,2024,1,1,Monday,00:15,97,32,9,138.0,19.4,32,22.5,74
2,2024-01-01 00:30:00,2024,1,1,Monday,00:30,98,41,7,146.0,19.6,41,17.5,78
3,2024-01-01 00:45:00,2024,1,1,Monday,00:45,88,31,12,131.0,17.6,31,30.0,79
4,2024-01-01 01:00:00,2024,1,1,Monday,01:00,56,28,12,96.0,11.2,28,30.0,69


In [44]:
df_waktu = df[['HARI','JAM']]
df_waktu = df_waktu.groupby(['HARI', 'JAM']).mean().round().reset_index()
df_waktu.head()

,HARI,JAM
0,Friday,00:00
1,Friday,00:15
2,Friday,00:30
3,Friday,00:45
4,Friday,01:00


In [45]:
all_df_outlier = []
all_df_cleaned = []

for i, row in df_waktu.iterrows():
    df_outlier,  df_cleaned = delete_outlier(df, row['HARI'], row['JAM'])
    all_df_outlier.append(df_outlier)
    all_df_cleaned.append(df_cleaned)

# gabungkan data
df_outlier_combined = pd.concat(all_df_outlier)
df_cleaned_combined = pd.concat(all_df_cleaned)

In [46]:
df_outlier_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
9794,2024-04-12 00:15:00,2024,4,12,Friday,00:15,114,53,11,178.0,22.8,53,27.5,103
9795,2024-04-12 00:30:00,2024,4,12,Friday,00:30,108,59,4,171.0,21.6,59,10.0,91
9796,2024-04-12 00:45:00,2024,4,12,Friday,00:45,119,54,6,179.0,23.8,54,15.0,93
9797,2024-04-12 01:00:00,2024,4,12,Friday,01:00,130,49,8,187.0,26.0,49,20.0,95
11141,2024-04-26 01:00:00,2024,4,26,Friday,01:00,43,102,158,NaN,8.6,102,395.0,506


In [47]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
384,2024-01-05,2024,1,5,Friday,00:00,30,65,112,207.0,6.0,65,280.0,351
1056,2024-01-12,2024,1,12,Friday,00:00,40,43,45,128.0,8.0,43,112.5,164
1728,2024-01-19,2024,1,19,Friday,00:00,35,41,45,121.0,7.0,41,112.5,160
2400,2024-01-26,2024,1,26,Friday,00:00,20,54,89,163.0,4.0,54,222.5,280
3072,2024-02-02,2024,2,2,Friday,00:00,23,44,60,127.0,4.6,44,150.0,199


In [48]:
df_outlier_combined = df_outlier_combined.sort_values(by='WAKTU').reset_index(drop=True)
df_cleaned_combined = df_cleaned_combined.sort_values(by='WAKTU').reset_index(drop=True)

In [49]:
df_cleaned_combined.to_excel('../Data/Preprocessing/Data Losarang/2024/3_clean_losarang_2024.xlsx', index=False)
df_outlier_combined.to_excel('../Data/Preprocessing/Data Losarang/2024/3_outliers_losarang_2024.xlsx', index=False)

In [50]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-01 00:00:00,2024,1,1,Monday,00:00,78,30,12,120.0,15.6,30,30.0,76
1,2024-01-01 00:15:00,2024,1,1,Monday,00:15,97,32,9,138.0,19.4,32,22.5,74
2,2024-01-01 00:45:00,2024,1,1,Monday,00:45,88,31,12,131.0,17.6,31,30.0,79
3,2024-01-01 01:15:00,2024,1,1,Monday,01:15,61,34,17,112.0,12.2,34,42.5,89
4,2024-01-01 01:30:00,2024,1,1,Monday,01:30,60,38,30,128.0,12.0,38,75.0,125


In [51]:
df_smp = df_cleaned_combined[['WAKTU','TAHUN', 'BULAN', 'TANGGAL', 'HARI','JAM','MOTOR','MOBIL','TRUK/BUS',
                              'SMP_MTR','SMP_MBL','SMP_TRK','SMP']]

In [52]:
df_smp.head(8)

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-01 00:00:00,2024,1,1,Monday,00:00,78,30,12,15.6,30,30.0,76
1,2024-01-01 00:15:00,2024,1,1,Monday,00:15,97,32,9,19.4,32,22.5,74
2,2024-01-01 00:45:00,2024,1,1,Monday,00:45,88,31,12,17.6,31,30.0,79
3,2024-01-01 01:15:00,2024,1,1,Monday,01:15,61,34,17,12.2,34,42.5,89
4,2024-01-01 01:30:00,2024,1,1,Monday,01:30,60,38,30,12.0,38,75.0,125
5,2024-01-01 01:45:00,2024,1,1,Monday,01:45,57,45,15,11.4,45,37.5,94
6,2024-01-01 02:00:00,2024,1,1,Monday,02:00,58,26,11,11.6,26,27.5,65
7,2024-01-01 02:15:00,2024,1,1,Monday,02:15,62,44,20,12.4,44,50.0,106


In [53]:
# Round down the WAKTU column to the nearest hour
df_smp_jam = df_smp.copy()
df_smp_jam['JAM'] = df_smp['WAKTU'].dt.floor('H')

# Drop the original WAKTU column if it's not needed anymore
df_smp_jam = df_smp_jam.drop(columns=['WAKTU'])

# Format the 'JAM' column to show only hour and minute
df_smp_jam['JAM'] = df_smp_jam['JAM'].dt.strftime('%H:%M')

# Group by the rounded-down hour and sum the SMP column
df_smp_jam = df_smp_jam.groupby(['TAHUN', 'BULAN', 'TANGGAL', 'HARI', 'JAM'], as_index=False).mean().round()

In [54]:
df_smp_jam.head(8)

,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024,1,1,Monday,00:00,88.0,31.0,11.0,18.0,31.0,28.0,76.0
1,2024,1,1,Monday,01:00,59.0,39.0,21.0,12.0,39.0,52.0,103.0
2,2024,1,1,Monday,02:00,55.0,31.0,13.0,11.0,31.0,33.0,76.0
3,2024,1,1,Monday,03:00,41.0,24.0,14.0,8.0,24.0,36.0,68.0
4,2024,1,1,Monday,04:00,45.0,21.0,9.0,9.0,21.0,22.0,53.0
5,2024,1,1,Monday,05:00,52.0,20.0,14.0,10.0,20.0,36.0,66.0
6,2024,1,1,Monday,06:00,96.0,32.0,24.0,19.0,32.0,60.0,111.0
7,2024,1,1,Monday,07:00,157.0,50.0,26.0,31.0,50.0,65.0,146.0


In [55]:
df_lhrt = df_smp_jam.copy()

In [56]:
df_lhrt = df_lhrt.drop(columns=['TAHUN','BULAN','TANGGAL'])

In [57]:
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Monday,00:00,88.0,31.0,11.0,18.0,31.0,28.0,76.0
1,Monday,01:00,59.0,39.0,21.0,12.0,39.0,52.0,103.0
2,Monday,02:00,55.0,31.0,13.0,11.0,31.0,33.0,76.0
3,Monday,03:00,41.0,24.0,14.0,8.0,24.0,36.0,68.0
4,Monday,04:00,45.0,21.0,9.0,9.0,21.0,22.0,53.0


In [58]:
df_lhrt  = df_lhrt.groupby(['HARI', 'JAM'], as_index=False).mean().round()
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Friday,00:00,63.0,59.0,95.0,13.0,59.0,237.0,308.0
1,Friday,01:00,59.0,62.0,101.0,12.0,62.0,251.0,325.0
2,Friday,02:00,58.0,57.0,89.0,12.0,57.0,222.0,291.0
3,Friday,03:00,49.0,54.0,83.0,10.0,54.0,206.0,271.0
4,Friday,04:00,45.0,50.0,76.0,9.0,50.0,190.0,250.0


In [59]:
len(df_lhrt)

168

In [60]:
df_lhrt.to_excel('../Data/Preprocessing/Data Losarang/2024/3_lhrt_losarang_2024.xlsx', index=False)